In [5]:
import json 
from pathlib import Path
import pandas as pd
import os
import re
from tqdm import tqdm
import cv2


In [6]:
def sort_key(filename, alphabetic_stem:str="graphene"):
    regex_pattern = "{}_(\d+)".format(alphabetic_stem)
    match = re.match(regex_pattern, filename)
    
    #match = re.match(r'graphene_(\d+)', filename)
    if match:
        return int(match.group(1))
    else:
        return filename

In [7]:
def get_flake_total_area(image_path: Path) -> int:
    img = cv2.imread(str(image_path))
    
    if len(img.shape)==2:
        h, w = img.shape
    elif len(img.shape)==3:
        h, w, _ = img.shape

    return h * w

In [8]:
directory = Path('./data_total_no_1/json_files/')
images = Path('./data_total_no_1/images/')
transport_output = Path('./data_total_no_1/transport_output/')
selected_keys = ['file_name', 'n_electrons', 'fermi_level_ev', "total_energy_eV","total_energy_eV_-1","total_energy_eV_+1", "IP_ev","EA_ev","band_gap_ev"]

# Lista per tenere traccia di tutti i dizionari da tutti i file JSON
data_list = []

files = [f for f in directory.iterdir() if f.suffix.lower() ==".json"]

sorted_files= sorted(files, key=lambda x: sort_key(x.name))

for file in tqdm(sorted_files):
    with open(str(file), 'r') as f:
        data = json.load(f)
        
        # Seleziona solo le chiavi desiderate
        selected_data = {key: data[key] for key in selected_keys}
    try:
        with open(str(transport_output.joinpath(f"{file.stem[:-4]}_e.json")), 'r') as f:
            data = json.load(f)
        
            selected_data["current"] = data["current"]
        
        selected_data["flake_total_area"] = get_flake_total_area(images.joinpath(f"{file.stem[:-4]}_opt.png"))
            
    except FileNotFoundError:
        continue
        
    data_list.append(selected_data)
        
        
df = pd.DataFrame(data_list)

df['file_name'] = df['file_name'].str.slice(0, -4)

# Esporta il DataFrame in un file CSV
df.to_csv('./data_total_no_1/xyz_files/target_graphene_dftb.csv', index=False)


100%|██████████| 8750/8750 [01:43<00:00, 84.83it/s] 
